# Check the manually constructed dataset of mining areas and ground truth masks

In [1]:
import geopandas as gpd
import leafmap
from shapely.ops import unary_union
from shapely.geometry import Point, mapping, box, shape
import shapely
import json
import os
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append("..")

os.getcwd()
os.chdir("..")
root = os.path.dirname(os.getcwd())
root = root + "/workspaces/mine-segmentation" # uncomment when running in Lightning Studios
root

'/teamspace/studios/this_studio/workspaces/mine-segmentation'

In [15]:
raw_dataset = root + "/data/raw/mining_tiles_with_masks.gpkg"
post_dataset = root + "/data/processed/mining_tiles_with_masks_and_bounding_boxes.gpkg"

# RAW DATASET

In [23]:
gpd.list_layers(raw_dataset)

,name,geometry_type
0,tiles,Polygon
1,maus_polygons,MultiPolygon
2,tang_polygons,Unknown


In [3]:
# Load the data
data = gpd.read_file(raw_dataset, layer="tiles")
print(data.shape)
data.head()

(1517, 9)


,tile_id,s2_tile_id,source_dataset,preferred_dataset,minetype1,minetype2,comment,timestamp,geometry
0,2805,S2A_MSIL2A_20191017T004711_R102_T54LWM_2020100...,maus,maus,Surface,Industrial,,2024-07-04 10:04:22.755,"POLYGON ((141.8445 -12.50898, 141.84391 -12.32..."
1,1612,S2B_MSIL2A_20191007T061729_R034_T42UXD_2020100...,tang,tang,Surface,Industrial,,2024-07-04 10:05:26.560,"POLYGON ((72.06065 52.32091, 72.07342 52.50476..."
2,2352,S2B_MSIL2A_20190707T021609_R003_T50LMR_2020100...,both,tang,Surface,Industrial,,2024-07-04 10:06:29.655,"POLYGON ((117.00981 -9.0093, 117.00981 -8.8240..."
3,1167,S2A_MSIL2A_20190424T082611_R021_T33JYF_2020110...,both,tang,Surface,Industrial,,2024-07-04 10:07:19.452,"POLYGON ((18.02702 -31.33978, 18.02113 -31.155..."
4,1107,S2A_MSIL2A_20180827T101021_R022_T32SMD_2020102...,both,tang,Surface,Industrial,,2024-07-04 10:09:21.708,"POLYGON ((8.36212 34.32496, 8.36072 34.50964, ..."


In [28]:
# check distribution in minetype1 column
data["minetype1"].value_counts()

minetype1
Surface    1060
Placer      147
Name: count, dtype: int64

In [29]:
# check distribution in source_dataset column
data.source_dataset.value_counts()

source_dataset
both    695
maus    392
tang    120
Name: count, dtype: int64

In [6]:
# filter out only tiles with valid surface mines
data = data[data["minetype1"].isin(["Surface", "Placer"]) & data["source_dataset"].isin(["maus", "tang", "both"])]
print(data.shape)
data.source_dataset.value_counts()

(1210, 9)


source_dataset
both    697
maus    393
tang    120
Name: count, dtype: int64

## Overlap

In [ ]:
# check how many tiles are overlapping with other tiles
data["overlaps"] = data["geometry"].apply(lambda x: data["geometry"].apply(lambda y: x.overlaps(y)).sum())
data["overlaps"].value_counts()

In [ ]:
# calculate overlap percentage of each tile
data["overlap_percentage"] = data["geometry"].apply(lambda x: data["geometry"].apply(lambda y: x.intersection(y).area/x.area).sum() - 1)
data["overlap_percentage"].hist(bins=50)

In [ ]:
# assign validation label to the tiles with overlap percentage less than 0.1
data["validation_eligible"] = data["overlap_percentage"].apply(lambda x: 1 if x < 0.1 else 0)
data["validation_eligible"].value_counts()

In [ ]:
# Create a Leaflet map
m = leafmap.Map()

style_eligible = {
    "stroke": True,
    "color": "green",
    "weight": 2,
    "opacity": 1,
    "fill": True,
    "fillColor": "green",
}

# change the color of the tiles with overlap percentage less than 0.1
m.add_gdf(data[data["validation_eligible"] == 1], layer_name="validation_eligible", style=style_eligible)

style_not_eligible = {
    "stroke": True,
    "color": "red",
    "weight": 2,
    "opacity": 1,
    "fill": True,
    "fillColor": "red",
}

# change the color of the tiles with overlap percentage more than 0.1
m.add_gdf(data[data["validation_eligible"] == 0], layer_name="validation_not_eligible", style=style_not_eligible)

# Display the map
m

### Group Overlapping Tiles


In [ ]:
import geopandas as gpd
import networkx as nx
from shapely.geometry import Polygon

# Assuming `data` is a GeoDataFrame with a 'geometry' column containing Polygon geometries

# Step 1: Create a graph
G = nx.Graph()

# Step 2: Add nodes
for idx, geom in data.iterrows():
    G.add_node(idx, geometry=geom.geometry)

# Step 3: Add edges for overlapping or touching geometries
for i, geom1 in data.iterrows():
    for j, geom2 in data.iterrows():
        if i != j and (geom1.geometry.overlaps(geom2.geometry) or geom1.geometry.touches(geom2.geometry)):
            G.add_edge(i, j)

# Step 4: Find connected components
connected_components = list(nx.connected_components(G))

# Step 5: Assign group IDs
group_id = 0
data['overlap_group'] = -1
for component in connected_components:
    for idx in component:
        data.at[idx, 'overlap_group'] = group_id
    group_id += 1

print(f"Number of connected components: {len(connected_components)}")

data.head()

In [ ]:
# Create a Leaflet map
m = leafmap.Map()
m.add_gdf(data)
m

### Plot groups by color

In [ ]:
import geopandas as gpd
import leafmap
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Step 2: Define styles
def get_group_color(group_id, num_groups):
    cmap = plt.get_cmap('tab20')  # Use a colormap with enough distinct colors
    norm = mcolors.Normalize(vmin=0, vmax=num_groups - 1)
    return mcolors.to_hex(cmap(norm(group_id)))

# Get the number of unique groups
num_groups = data['overlap_group'].nunique()

# Step 3: Add tiles to the map
m = leafmap.Map()

for group_id in data['overlap_group'].unique():
    group_data = data[data['overlap_group'] == group_id]
    color = get_group_color(group_id, num_groups)
    style = {
        "stroke": True,
        "color": color,
        "weight": 2,
        "opacity": 1,
        "fill": True,
        "fillColor": color,
        "fillOpacity": 0.5,
    }
    m.add_gdf(group_data, layer_name=f"group_{group_id}", style=style)

# Display the map
m

## Global distribution of tiles

In [ ]:
# Using Markers on leafmap
def plot_tiles_on_basemap(gdf):
    import folium

    m = folium.Map()

    # Iterate over the tiles and add markers to the map
    for _, row in gdf.iterrows():
        lat = row.geometry.centroid.y
        lon = row.geometry.centroid.x
        folium.Marker([lat, lon]).add_to(m)

    # Display the map
    return m

plot_tiles_on_basemap(data)

# PROCESSED DATASET

In [30]:
# list layernames 
gpd.list_layers(post_dataset)

,name,geometry_type
0,maus_polygons,MultiPolygon
1,tang_polygons,Unknown
2,maus_bboxes,MultiPolygon
3,tang_bboxes,MultiPolygon
4,preferred_polygons,MultiPolygon
5,preferred_bboxes,MultiPolygon
6,tiles,Polygon


In [27]:
# load the processed dataset
tiles = gpd.read_file(post_dataset, layer="tiles")
maus_poly = gpd.read_file(post_dataset, layer="maus_polygons")
tang_poly = gpd.read_file(post_dataset, layer="tang_polygons")
preferred_poly = gpd.read_file(post_dataset, layer="preferred_polygons")
maus_bbox = gpd.read_file(post_dataset, layer="maus_bboxes")
tang_bbox = gpd.read_file(post_dataset, layer="tang_bboxes")
preferred_bbox = gpd.read_file(post_dataset, layer="preferred_bboxes")
tiles

,tile_id,s2_tile_id,source_dataset,preferred_dataset,minetype1,minetype2,comment,timestamp,split,geometry
0,2805,S2A_MSIL2A_20191017T004711_R102_T54LWM_2020100...,maus,maus,Surface,Industrial,,2024-07-04 10:04:22.755,train,"POLYGON ((141.8445 -12.50898, 141.84391 -12.32..."
1,1612,S2B_MSIL2A_20191007T061729_R034_T42UXD_2020100...,tang,tang,Surface,Industrial,,2024-07-04 10:05:26.560,train,"POLYGON ((72.06065 52.32091, 72.07342 52.50476..."
2,2352,S2B_MSIL2A_20190707T021609_R003_T50LMR_2020100...,both,tang,Surface,Industrial,,2024-07-04 10:06:29.655,train,"POLYGON ((117.00981 -9.0093, 117.00981 -8.8240..."
3,1167,S2A_MSIL2A_20190424T082611_R021_T33JYF_2020110...,both,tang,Surface,Industrial,,2024-07-04 10:07:19.452,train,"POLYGON ((18.02702 -31.33978, 18.02113 -31.155..."
4,1107,S2A_MSIL2A_20180827T101021_R022_T32SMD_2020102...,both,tang,Surface,Industrial,,2024-07-04 10:09:21.708,train,"POLYGON ((8.36212 34.32496, 8.36072 34.50964, ..."
...,...,...,...,...,...,...,...,...,...,...
1509,630,S2B_MSIL2A_20190921T143749_R096_T20NNN_2020100...,rejected,none,Placer,Artisanal,cloudy image,2024-08-27 16:29:40.578,None,"POLYGON ((-62.32414 6.65726, -62.32388 6.84251..."
1510,2006,S2B_MSIL2A_20190729T025549_R032_T48MZB_2020110...,rejected,none,Placer,Artisanal,cloudy image,2024-08-27 16:31:52.014,None,"POLYGON ((108.17512 -3.17611, 108.17561 -2.991..."
1511,861,S2B_MSIL2A_20190912T141049_R110_T21NXF_2020100...,rejected,none,Placer,Artisanal,inaccurate polygons,2024-08-27 16:35:28.938,None,"POLYGON ((-55.49122 4.6572, -55.49082 4.84241,..."
1512,1654,S2B_MSIL2A_20190330T050659_R019_T44QLJ_2020100...,both,tang,Surface,Industrial,,2024-08-27 16:38:46.515,train,"POLYGON ((79.34968 21.15852, 79.34762 21.34348..."


Construct one coherent dataset with the preferred mask

## Visualize the dataset

In [ ]:
ROW = 6

m = leafmap.Map(center = (tiles.iloc[ROW:ROW+1].geometry.centroid.y.values[0], tiles.iloc[ROW:ROW+1].geometry.centroid.x.values[0]), zoom = 12)
# add satellite
m.add_basemap("SATELLITE")

style_tile = {
    "color": "orange",
    "fillColor": "orange",
    "fillOpacity": 0.0,
}

style_bbox = {
    "color": "green",
    "fillColor": "green",
    "fillOpacity": 0.0,
}

m.add_gdf(tiles.iloc[ROW:ROW+1], layer_name="tiles", style=style_tile)
m.add_gdf(preferred_poly.iloc[ROW:ROW+1], layer_name="masks")
m.add_gdf(preferred_bbox.iloc[ROW:ROW+1], layer_name="bboxes", style=style_bbox)
m

## FILTERED DATASET
This includes the train/test/val split

In [37]:
# only tiles with split
data = tiles[tiles["split"].notnull()]
print(data.shape)
data.head(2)

(1207, 10)


,tile_id,s2_tile_id,source_dataset,preferred_dataset,minetype1,minetype2,comment,timestamp,split,geometry
0,2805,S2A_MSIL2A_20191017T004711_R102_T54LWM_2020100...,maus,maus,Surface,Industrial,,2024-07-04 10:04:22.755,train,"POLYGON ((141.8445 -12.50898, 141.84391 -12.32..."
1,1612,S2B_MSIL2A_20191007T061729_R034_T42UXD_2020100...,tang,tang,Surface,Industrial,,2024-07-04 10:05:26.560,train,"POLYGON ((72.06065 52.32091, 72.07342 52.50476..."


In [38]:
# Provide unique value counts for categorical columns
categorical_columns = ['source_dataset', 'preferred_dataset', 'minetype1', 'minetype2', 'split']
for col in categorical_columns:
    print(f"Unique values in {col}:")
    print(data[col].value_counts())

Unique values in source_dataset:
source_dataset
both    695
maus    392
tang    120
Name: count, dtype: int64
Unique values in preferred_dataset:
preferred_dataset
tang    619
maus    588
Name: count, dtype: int64
Unique values in minetype1:
minetype1
Surface    1060
Placer      147
Name: count, dtype: int64
Unique values in minetype2:
minetype2
Industrial    975
Artisanal     232
Name: count, dtype: int64
Unique values in split:
split
train    922
val      201
test      84
Name: count, dtype: int64


## Visualize

In [ ]:
# Create a Leaflet map
m = leafmap.Map()

# add satellite layer
# m.add_basemap("SATELLITE")

style_train = {
    "stroke": True,
    "color": "green",
    "weight": 2,
    "opacity": 1,
    "fill": True,
    "fillColor": "green",
}

style_val = {
    "stroke": True,
    "color": "blue",
    "weight": 2,
    "opacity": 1,
    "fill": True,
    "fillColor": "blue",
}

style_test = {
    "stroke": True,
    "color": "red",
    "weight": 2,
    "opacity": 1,
    "fill": True,
    "fillColor": "red",
}

# Add train tiles to the map
m.add_gdf(data[data["split"] == "train"], layer_name="train", style=style_train)

# Add val tiles to the map
m.add_gdf(data[data["split"] == "val"], layer_name="val", style=style_val)

# Add test tiles to the map
m.add_gdf(data[data["split"] == "test"], layer_name="test", style=style_test)

# Display the map
m

In [ ]:
# Create a Leaflet map
m = leafmap.Map()

# Add the centroids of each test tile as markers
test_tiles = data[data["split"] == "test"]
for _, row in test_tiles.iterrows():
    lat = row.geometry.centroid.y
    lon = row.geometry.centroid.x
    m.add_marker(location=[lat, lon])

m

## Check Train Test Splits

In [22]:
# train test split for the combined dataset
print(f"The train/val/test split for the combined dataset is: {data['split'].value_counts(normalize=True)}")

# train test split for the maus dataset
print(f"The train/val/test split for the maus dataset is: {data[data['source_dataset'].isin(['maus', 'both'])]['split'].value_counts(normalize=True)}")

# train test split for the tang dataset
print(f"The train/val/test split for the tang dataset is: {data[data['source_dataset'].isin(['tang', 'both'])]['split'].value_counts(normalize=True)}")


The train/val/test split for the combined dataset is: split
train    0.763877
val      0.166529
test     0.069594
Name: proportion, dtype: float64
The train/val/test split for the maus dataset is: split
train    0.758050
val      0.164673
test     0.077277
Name: proportion, dtype: float64
The train/val/test split for the tang dataset is: split
train    0.743558
val      0.153374
test     0.103067
Name: proportion, dtype: float64
